## Retrieve arguments generated from scratch (no vector database)

In [3]:
# from google.colab import userdata
# openai_api_key = userdata.get('openai_api_key')

In [2]:
# Import necessary libraries
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv('OPENAI_API_KEY')


In [4]:
!pip install pymupdf -q

In [8]:
import requests
import json
from pprint import pp
import fitz # PyMuPDF library

### openai api GPT LLM model

In [6]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=1000):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

### openai api GPT embedding model

In [10]:
def get_embeddings(input, dimensions = 1536, model="text-embedding-3-small"):
  payload = { "input": input, "model": model, "dimensions": dimensions }
  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/embeddings', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["data"][0]["embedding"]
  else :
    return obj["error"]

### OCR PDF text extraction 

國立臺灣大學社會科學院經濟學系 - 以太幣價格與總體經濟變數之實證分析

In [13]:
import fitz  # PyMuPDF

def extract_text_from_pdf_with_fitz(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page in document:
        text += page.get_text()
    document.close()
    return text


pdf_path = 'ntu-111-2.pdf'
content = extract_text_from_pdf_with_fitz(pdf_path)
print(content)


doi:10.6342/NTU202301064
國立臺灣大學社會科學院經濟學系
碩士論文
Department of Economics
College of Social Sciences
National Taiwan University
Master Thesis
以太幣價格與總體經濟變數之實證分析
An Empirical Analysis of Relationship between Macroeconomics Variables and
Ethereum Price
黃耕偉
King-Ui Ng
指導教授: 謝德宗 博士
Advisor: Der-Tzon Hsieh, Ph.D.
中華民國112年6月
June, 2023
doi:10.6342/NTU202301064
摘要
高波動性的加密貨幣市場吸引許多人的注意,2021年 NFT 與元宇宙的熱潮更是大幅推
升加密貨幣的價格。 現今加密貨幣已是大眾關注的焦點。
自比特幣暴漲的2017年以來, 學界也開始對加密貨幣產生興趣。 惟過去相關研究大多
以比特幣為主題, 討論以太幣價格與總體經濟變數如何互動的文獻甚少, 並多未考慮 Defi
總鎖倉量對以太幣價格的影響。 以太幣是以太坊的原生代幣, 而以太坊與比特幣最大的差
異之一在於以太坊可以在區塊上執行應用程式, Defi 總鎖倉量則顯示這些應用程式的熱門
程度, 因此 Defi 總鎖倉量對以太幣價格的影響必須考慮。
本文使用向量誤差修正模型分析以太幣價格與總體經濟變數之關係。 模型中變數包含
以太幣價格、比特幣價格、Defi 總鎖倉量、布蘭特原油期貨價格、黃金期貨價格、美元指數、
日圓兌美元匯率、人民幣兌美元匯率、標準普爾500指數與 VIX 恐慌指數。 實證發現變數
之間存在長期均衡關係。 同時也透過 Granger causality 檢視總體經濟變數是否能夠預測
以太幣價格, 以及利用衝擊反應函數分析各個解釋變數衝擊對以太幣價格的影響。
關鍵詞: 以太幣; 比特幣; 總鎖倉量; 向量誤差修正模型; 衝擊反應函數
i
doi:10.6342/NTU202301064
Abstract
The cryptocurrency market, characterized by its high volatility, has attracted 

## Compute the token for the entire document

In [11]:
!pip install tiktoken -q

In [14]:
import tiktoken
tokenizer = tiktoken.get_encoding("cl100k_base")

In [15]:
num_tokens = len(tokenizer.encode(content))
print(num_tokens)

52907


## chunks split with documents

example from scratch

In [17]:
# 參考自 https://platform.openai.com/docs/tutorials/web-qa-embeddings
import re
def split_into_many(text, max_tokens = 800):

    # 使用 re.split 來拆分字符串，並用 filter 函數去除空字符串
    pattern = r'[。.]' # 中文。和英文.
    sentences = list(filter(None, re.split(pattern, text)))

    # Get the number of tokens for each sentence
    n_tokens = [len(tokenizer.encode(" " + sentence)) for sentence in sentences]

    chunks = []
    tokens_so_far = 0
    chunk = []

    # Loop through the sentences and tokens joined together in a tuple
    for sentence, token in zip(sentences, n_tokens):

        # If the number of tokens so far plus the number of tokens in the current sentence is greater
        # than the max number of tokens, then add the chunk to the list of chunks and reset
        # the chunk and tokens so far
        if tokens_so_far + token > max_tokens:
            chunks.append(". ".join(chunk) + ".")
            chunk = []
            tokens_so_far = 0

        # If the number of tokens in the current sentence is greater than the max number of
        # tokens, go to the next sentence
        if token > max_tokens:
            continue

        # Otherwise, add the sentence to the chunk and add the number of tokens to the total
        chunk.append(sentence)
        tokens_so_far += token + 1

    return chunks


In [18]:
docs = split_into_many(content)
docs


['doi:10. 6342/NTU202301064\n國立臺灣大學社會科學院經濟學系\n碩士論文\nDepartment of Economics\nCollege of Social Sciences\nNational Taiwan University\nMaster Thesis\n以太幣價格與總體經濟變數之實證分析\nAn Empirical Analysis of Relationship between Macroeconomics Variables and\nEthereum Price\n黃耕偉\nKing-Ui Ng\n指導教授: 謝德宗 博士\nAdvisor: Der-Tzon Hsieh, Ph. D. \n中華民國112年6月\nJune, 2023\ndoi:10. 6342/NTU202301064\n摘要\n高波動性的加密貨幣市場吸引許多人的注意,2021年 NFT 與元宇宙的熱潮更是大幅推\n升加密貨幣的價格.  現今加密貨幣已是大眾關注的焦點. \n自比特幣暴漲的2017年以來, 學界也開始對加密貨幣產生興趣.  惟過去相關研究大多\n以比特幣為主題, 討論以太幣價格與總體經濟變數如何互動的文獻甚少, 並多未考慮 Defi\n總鎖倉量對以太幣價格的影響.  以太幣是以太坊的原生代幣, 而以太坊與比特幣最大的差\n異之一在於以太坊可以在區塊上執行應用程式, Defi 總鎖倉量則顯示這些應用程式的熱門\n程度, 因此 Defi 總鎖倉量對以太幣價格的影響必須考慮. \n本文使用向量誤差修正模型分析以太幣價格與總體經濟變數之關係.  模型中變數包含\n以太幣價格、比特幣價格、Defi 總鎖倉量、布蘭特原油期貨價格、黃金期貨價格、美元指數、\n日圓兌美元匯率、人民幣兌美元匯率、標準普爾500指數與 VIX 恐慌指數.  實證發現變數\n之間存在長期均衡關係.',
 ' 同時也透過 Granger causality 檢視總體經濟變數是否能夠預測\n以太幣價格, 以及利用衝擊反應函數分析各個解釋變數衝擊對以太幣價格的影響. \n關鍵詞: 以太幣; 比特幣; 總鎖倉量; 向量誤差修正模型; 衝擊反應函數\ni\ndoi:10. 6342/NTU202301064\nAbstract\nThe cryptocurrency market

In [19]:
len(docs)

76

## Cosine similarity 

In [21]:
embeddings = [] # 這個存所有當作索引的向量
for doc in docs:
  embeddings.append( get_embeddings(doc) )

In [22]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 參數 list_of_doc_vectors 是所有文件的 embeddings 向量
# 參數 query_vector 是查詢字串的 embedding 向量
# 參數 top_k 是回傳的比數
def get_top_k_indices(list_of_doc_vectors, query_vector, top_k):
  # 轉成 numpy arrays
  list_of_doc_vectors = np.array(list_of_doc_vectors)
  query_vector = np.array(query_vector)

  # 計算 cosine similarities
  similarities = cosine_similarity(query_vector.reshape(1, -1), list_of_doc_vectors).flatten()

  # 根據 cosine similarity 排序
  sorted_indices = np.argsort(similarities)[::-1]

  # 取出 top K 的索引數字
  top_k_indices = sorted_indices[:top_k]

  return top_k_indices


In [24]:
def search_similar(embeddings, query, n=3):
    query_embedding = get_embeddings(query)

    best_indexes = get_top_k_indices(embeddings, query_embedding, n)
    best_k_chunks = [docs[i] for i in best_indexes]

    return best_k_chunks

## QA check with the relevant documents

In [31]:
question = "乙太坊和比特幣的差異，給我摘要和結論"
chunks = search_similar(embeddings, question)

In [32]:
context = ''
for chunk in chunks:
  context += f"{chunk} \n----------------\n"

In [33]:
print(context)

 驗證交易者能夠獲得以太幣作為獎勵, 而這筆費用由進行交易的人支付,
以太坊便是以這樣的方式維持運作.  一直以來, 比特幣是加密貨幣中公認的王者.  近期由
1
doi:10. 6342/NTU202301064
第 1 章 導論
於以太坊2. 0升級改善了以太坊區塊鏈上的基礎建設, 開始出現以太幣是否能夠超越比特
幣成為加密貨幣龍頭的爭論.  如圖1. 1, 透過歷史走勢可以發現, 比特幣價格的波動相當大. 
2020年3月低點到2021年11月最高點, 兩年間比特幣飆漲將近17倍. 
圖 1. 1: 比特幣走勢圖
由於比特幣是加密貨幣之中的指標性商品, 近年來開始有文獻研究比特幣價格波動的
成因. 
然而如圖1. 2所示, 在相同時間區間內, 以太幣的表現更具爆發力, 漲幅53倍. 
圖 1. 2: 以太幣走勢圖
但是在過往的加密貨幣相關文獻之中, 以太幣的研究卻少之又少.  討論以太幣價格波動
形成原因的研究更是鳳毛麟角.  有鑑於此, 本文探討的主題係為以太幣與總體經濟變數之
間的互動關係, 以期達到拋磚引玉的效果. 
2
doi:10. 6342/NTU202301064
第 1 章 導論
1. 2
研究目的
過往已有文獻探討比特幣價格波動形成原因, 但是關於以太幣之討論甚少.  且在以太幣
相關文獻中, 幾乎沒有討論 Defi 總鎖倉量 (total value locked, TVL) 對於以太幣價格波動
的影響. 
----------------
 礦工透過電腦運算驗證交易資
訊, 成功驗證可取得比特幣做為獎勵, 這樣的行為稱為挖礦.  然而在眾多的礦工之中要挖到
礦其實不容易.  比特幣系統是利用工作量證明 (Proof-of-Work, PoW) 機制決定哪個礦工
5
doi:10. 6342/NTU202301064
第 2 章 文獻回顧
可以得到挖礦的收益.  交易記錄的驗證方式是解一道數學題, 且解題的唯一方法就是不斷
的試誤 (trial and error).  而挖礦的過程就是一場競速, 搶先解出答案的礦工可以得到比特
幣做為獎勵.  一旦一筆交易經過驗證, 其他人要驗算就很容易了.  因此只有第一個解出答
案的礦工可以收到比特幣.  但是比特幣具有總量限制, 最多只會發行2100萬枚比特幣.  每
當 「開採」 出21萬個區塊,

## prompt integration

In [34]:
prompt= f'''
Answer the question based on the context below,
and if the question can't be answered based on the context, say "I don't know"

Context:

{context}

Question: ```{question}```
Answer:'''

print(prompt)


Answer the question based on the context below,
and if the question can't be answered based on the context, say "I don't know"

Context:

 驗證交易者能夠獲得以太幣作為獎勵, 而這筆費用由進行交易的人支付,
以太坊便是以這樣的方式維持運作.  一直以來, 比特幣是加密貨幣中公認的王者.  近期由
1
doi:10. 6342/NTU202301064
第 1 章 導論
於以太坊2. 0升級改善了以太坊區塊鏈上的基礎建設, 開始出現以太幣是否能夠超越比特
幣成為加密貨幣龍頭的爭論.  如圖1. 1, 透過歷史走勢可以發現, 比特幣價格的波動相當大. 
2020年3月低點到2021年11月最高點, 兩年間比特幣飆漲將近17倍. 
圖 1. 1: 比特幣走勢圖
由於比特幣是加密貨幣之中的指標性商品, 近年來開始有文獻研究比特幣價格波動的
成因. 
然而如圖1. 2所示, 在相同時間區間內, 以太幣的表現更具爆發力, 漲幅53倍. 
圖 1. 2: 以太幣走勢圖
但是在過往的加密貨幣相關文獻之中, 以太幣的研究卻少之又少.  討論以太幣價格波動
形成原因的研究更是鳳毛麟角.  有鑑於此, 本文探討的主題係為以太幣與總體經濟變數之
間的互動關係, 以期達到拋磚引玉的效果. 
2
doi:10. 6342/NTU202301064
第 1 章 導論
1. 2
研究目的
過往已有文獻探討比特幣價格波動形成原因, 但是關於以太幣之討論甚少.  且在以太幣
相關文獻中, 幾乎沒有討論 Defi 總鎖倉量 (total value locked, TVL) 對於以太幣價格波動
的影響. 
----------------
 礦工透過電腦運算驗證交易資
訊, 成功驗證可取得比特幣做為獎勵, 這樣的行為稱為挖礦.  然而在眾多的礦工之中要挖到
礦其實不容易.  比特幣系統是利用工作量證明 (Proof-of-Work, PoW) 機制決定哪個礦工
5
doi:10. 6342/NTU202301064
第 2 章 文獻回顧
可以得到挖礦的收益.  交易記錄的驗證方式是解一道數學題, 且解題的唯一方法就是不斷
的試誤 (trial and error).

## embedding to LLM model

In [35]:
result = get_completion([ {"role": "user", "content": prompt }], model="gpt-4-turbo-preview")
print(result)

乙太坊和比特幣的主要差異在於它們的基本用途和技術架構。比特幣被設計為一種對等式的電子現金系統，主要用於交易和價值儲存，其使用區塊鏈技術紀錄交易。比特幣的挖礦過程依賴於工作量證明（Proof-of-Work, PoW）機制，礦工通過解決數學問題來驗證交易並獲得比特幣作為獎勵。比特幣有總量限制，最多只會發行2100萬枚。

相比之下，乙太坊是在比特幣的基礎上建立的，由Vitalik Buterin推出，旨在擴展加密貨幣的應用範圍。乙太坊不僅是一種加密貨幣（以太幣），更是一個支持智能合約和去中心化應用（DApps）的平台。這使得乙太坊像是一台全球性的去中心化電腦，能夠執行各種任務和金融服務，從而創建了一個豐富的去中心化金融（DeFi）生態系統。以太幣在乙太坊平台上用於支付交易費用和計算服務費用，這些費用被稱為燃料費（gas fee）。

總結來說，比特幣主要作為一種數字貨幣，專注於提供安全的價值儲存和轉移。而乙太坊則提供了一個更廣泛的應用平台，支持智能合約和去中心化應用，使其不僅限於貨幣的功能，而是創造了一個去中心化的應用生態系統。
